# 1. 開発実習環境セットアップ

# 1.1. Pythonのパッケージのインストール

In [ ]:
!pip install openai==0.27.8 
!pip install langchain==0.0.240
!pip install PyMuPDF==1.22.5
!pip install tiktoken==0.4.0
!pip install faiss-cpu==1.7.4

## 1.2. Pythonのモジュールのインポート

In [ ]:
import os
import json
from dotenv import load_dotenv

# OpenAI
import openai

# LangChain
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Azure OpenAI Service
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI

## 1.3. 環境変数の設定他

.envファイルから環境変数をロードします。各環境にあわせて.env.templateを基に.evnファイルを作成してください。

In [ ]:
load_dotenv()

In [ ]:
print(os.getenv("OPENAI_API_VERSION"))

Azure OpenAI ServiceのAPIを実行する際、デバッグレベルのログを出力するように設定します。

In [ ]:
openai.log = "debug"

# 2. PDFファイルからベクトルデータベースのインデックス作成

## 2-1. PDFファイルを読み込み、分割

埋め込みモデルのinputの上限を超えず、情報源として役に立つサイズに分割します。

In [ ]:
pdf_file_path = "./pdf/IBM_2021_ESG_Report.pdf"
loader = PyMuPDFLoader(pdf_file_path)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    separator="\n",
    chunk_size=2000,
    chunk_overlap=200,
    disallowed_special=(),
)
pages = loader.load_and_split(text_splitter=text_splitter)

PDFファイルを読み込んだ結果を確認してみます。

In [ ]:
type_of_page = type(pages[0])
page_count = len(pages)

print("pageの型:", type_of_page)
print("ページ数:", page_count)

1ページ分のデータを参照してみます。

In [ ]:
page = pages[11]
formatted_data = json.dumps(page.__dict__, indent=4, ensure_ascii=False)
print(formatted_data)


In [ ]:
# ページ内容
result = page.page_content

# ページ内容を見やすくするため、50文字ずつに改行して表示
line_length = 50
for i in range(0, len(result), line_length):
    print(result[i:i+line_length])

## 2.2. 埋め込みモデル（embedding）のインスタンス作成

埋め込みモデルには、Azure OpenAI Serviceを使います。

In [ ]:
EMBE_DEPLOYMENT_NAME = os.getenv("EMBE_DEPLOYMENT_NAME")
embeddings = OpenAIEmbeddings(deployment=EMBE_DEPLOYMENT_NAME, chunk_size=1)

埋め込みモデルを使って、テキストをベクトル化してみます。

In [ ]:
text = "これは埋め込みのテストです。"
embeddings.embed_query(text)

## 2-3. 埋め込みモデルを使って、PDFファイルからインデックスを作成する

ベクトルデータベースには、FAISSを使います。

In [ ]:
db = FAISS.from_documents(pages, embeddings)

検索してみます。

In [ ]:
query = "IBMは気候変動のリスクにどのように取り組んでいますか？"

docs = db.similarity_search(query)
for doc in docs:
    print("page", doc.metadata["page"], doc)

検索結果はデフォルトで4ドキュメントを含みます。それぞれ、PDFファイルのメタデータを持っています。

In [ ]:
def document_to_dict(document):
    return document.__dict__

formatted_data = json.dumps(docs[0], default=document_to_dict, indent=4, ensure_ascii=False)
print(formatted_data)

インデックスはローカルに保存することができます。

In [ ]:
index_path = "./index/IBM_2021_ESG_Report"
db.save_local(index_path)

保存したインデックスをロードすることができます。

In [ ]:
db = FAISS.load_local(index_path, embeddings)

# 3. Retrieval QAチェーンの作成

## 3.1. LLMのインスタンス作成を作成する

LLMにAzure OpenAI Serviceを使います。

In [ ]:
CHAT_DEPLOYMENT_NAME = os.getenv("CHAT_DEPLOYMENT_NAME")

llm = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT_NAME,
    temperature=0.5,)

## 3.2. Retrieval QAチェーンを作成する

retriever（インデックス内の文書や埋め込みを検索するためのメソッド）とLLMを使って、Retrieval QAチェーンを生成します。

In [ ]:
retriever = db.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True)

# 4. QA実施

Retrieval QAチェーンを使って、QAします。

In [ ]:
query = "IBMの温室効果ガス排出量削減目標を教えてください。"
response = qa_chain({"query": query})

回答を見やすく表示します。

In [ ]:
# レスポンスの回答部分
result = response["result"]

# 回答を見やすくするため、50文字ずつに改行して表示
line_length = 50
for i in range(0, len(result), line_length):
    print(result[i:i+line_length])

レスポンスを確認してみます。

In [ ]:
def document_to_dict(document):
    return document.__dict__

formatted_data = json.dumps(response, default=document_to_dict, indent=4, ensure_ascii=False)
print(formatted_data)